# Mapboxgl Python Library for location data visualization

https://github.com/mapbox/mapboxgl-jupyter


# Legend Styles and Controls

| Property  | Description  | Example |
|:--------- |:-------------|:--------|
| legend | controls visibility of map legend | True |
| legend_layout | controls orientation of map legend | 'horizontal' |
| legend_style | reserved for future custom CSS loading | '' |
| legend_gradient | boolean to determine appearance of legend keys; takes precedent over legend_key_shape | False |
| legend_fill | string background color for legend | 'white' |
| legend_header_fill | string background color for legend header (in vertical layout) | 'white' |
| legend_text_color | string color for legend text | '#6e6e6e' |
| legend_text_numeric_precision | decimal precision for numeric legend values | 0 |
| legend_title_halo_color | color of legend title text halo | 'white' |
| legend_key_shape | shape of the legend item keys, default varies by viz type; one of square, contiguous_bar, rounded-square, circle, line | 'square' |
| legend_key_borders_on | boolean for whether to show/hide legend key borders | False |

## Create a visualization from example data file

In [ ]:
import pandas as pd
import os
from mapboxgl.utils import *
from mapboxgl.viz import *

# Set Mapbox Acces Token; Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

# Load data from sample csv
data_url = 'https://raw.githubusercontent.com/mapbox/mapboxgl-jupyter/master/examples/data/points.csv'
df = pd.read_csv(data_url).round(3)

# Create a geojson Feature Collection from the current dataframe
geodata = df_to_geojson(df, 
                        properties=['Avg Medicare Payments', 'Avg Covered Charges', 'date'], 
                        lat='lat', 
                        lon='lon', 
                        precision=3)

# Generate data breaks using numpy quantiles and color stops from colorBrewer
measure = 'Avg Medicare Payments'
color_breaks = [round(df[measure].quantile(q=x*0.1), 2) for x in range(1,9)]
color_stops = create_color_stops(color_breaks, colors='YlOrRd')

# Create the viz from the dataframe
viz = CircleViz(geodata, 
                access_token=token, 
                color_property='Avg Medicare Payments',
                color_stops=color_stops,
                radius=2.5,
                stroke_width=0.2,
                center=(-95, 40),
                zoom=2.5,
                below_layer='waterway-label',
                height='300px')

# Show the viz
viz.show()

## Change the legend style to a horizontal contiguous bar

In [ ]:
viz.legend = True
viz.legend_layout = 'horizontal'
viz.legend_key_borders_on = True
viz.legend_key_shape = 'contiguous-bar'
viz.legend_text_numeric_precision = 0
viz.legend_key_borders_on = False
viz.show()

## Gradient legend (assumes linear color gradient)

In [ ]:
viz.legend_gradient = True
viz.show()

## Change the viz legend colors

In [ ]:
viz.legend_layout = 'vertical'
viz.legend_gradient = False
viz.legend_fill = '#f0f0ef'
viz.legend_text_color = '#000000'
viz.legend_header_fill = '#819092'
viz.legend_key_borders_on = False
viz.legend_title_halo_color = '#777'
viz.legend_key_shape = 'rounded-square'
viz.show()

## Update legend to match Mapbox Dark-v9 style

In [ ]:
# Map settings
viz.style='mapbox://styles/mapbox/dark-v9?optimize=true'
viz.label_color = 'hsl(0, 0%, 70%)'
viz.label_halo_color = 'hsla(0, 0%, 10%, 0.75)'
viz.height = '400px'

# Legend settings
viz.legend_gradient = False
viz.legend_fill = '#343332'
viz.legend_header_fill = '#343332'
viz.legend_text_color = 'hsl(0, 0%, 70%)'
viz.legend_key_borders_on = False
viz.legend_title_halo_color = 'hsla(0, 0%, 10%, 0.75)'

# Render map
viz.show()

## Default legend for a graduated circle viz

In [ ]:
# Generate data breaks and color stops from colorBrewer
measure_color = 'Avg Covered Charges'
color_breaks = [round(df[measure_color].quantile(q=x*0.1), 2) for x in range(2, 10)]
color_stops = create_color_stops(color_breaks, colors='Blues')

# Generate radius breaks from data domain and circle-radius range
measure_radius = 'Avg Medicare Payments'
radius_breaks = [round(df[measure_radius].quantile(q=x*0.1), 2) for x in range(2,10)]
radius_stops = create_radius_stops(radius_breaks, 0.5, 10)

# Create the viz
viz2 = GraduatedCircleViz(geodata, 
                          access_token=token,
                          color_property='Avg Covered Charges',
                          color_stops=color_stops,
                          radius_property='Avg Medicare Payments',
                          radius_stops=radius_stops,
                          stroke_color='black',
                          stroke_width=0.5,
                          center=(-95, 40),
                          zoom=2.5,
                          opacity=0.75,
                          below_layer='waterway-label',
                          height='300px')
viz2.show()

## Variable Radius Legend for a graduated circle viz

In [ ]:
# Modify the viz
viz2.legend_layout = 'horizontal'
viz2.legend_text_numeric_precision = 0

# Switch to a legend based on the radius property
viz2.legend_function = 'radius'

# Variable radius legend uses MapViz.color_default to set legend item color
viz2.color_default = '#0d3d79'

# Show updated viz
viz2.show()

## Default legend for a clustered circle map

In [ ]:
# Create a clustered circle map
color_stops = create_color_stops([1, 10, 25, 50, 75, 100], colors='YlOrRd')

viz3 = ClusteredCircleViz(geodata, 
                          access_token=token,
                          color_stops=color_stops,
                          stroke_color='black',
                          radius_stops=[[1,5], [10, 10], [50, 15], [100, 20]],
                          radius_default=2,
                          cluster_maxzoom=10,
                          cluster_radius=30,
                          label_size=12,
                          opacity=0.9,
                          center=(-95, 40),
                          zoom=2.5,
                          height='300px')
viz3.show()

## Default legend for choropleth viz

In [ ]:
# create choropleth from polygon features stored as GeoJSON
viz4 = ChoroplethViz('https://raw.githubusercontent.com/mapbox/mapboxgl-jupyter/master/examples/data/us-states.geojson', 
                     color_property='density',
                     color_stops=create_color_stops([0, 50, 100, 500, 1500], colors='YlOrRd'),
                     color_function_type='interpolate',
                     line_stroke='--',
                     line_color='rgb(128,0,38)',
                     line_width=1,
                     opacity=0.8,
                     center=(-96, 37.8),
                     zoom=2.5,
                     below_layer='waterway-label',
                     height='300px')
viz4.show()

## Change legend to vertical contiguous bar

In [ ]:
viz4.legend_layout = 'vertical'
viz4.legend_key_borders_on = False
viz4.legend_key_shape = 'contiguous-bar'
viz4.show()

## Default legend for linestring viz

In [ ]:
viz5 = LinestringViz([{"elevation": x} for x in range(0, 21000, 10)], 
                     vector_url='mapbox://mapbox.mapbox-terrain-v2',
                     vector_layer_name='contour',
                     vector_join_property='ele',
                     data_join_property='elevation',
                     color_property='elevation',
                     color_stops=create_color_stops([0, 50, 100, 200, 300], colors='YlOrRd'),
                     line_width_stops=create_numeric_stops([0, 50, 100, 200, 300], 0.1, 4),
                     line_width_property='elevation',
                     line_width_function_type='interpolate',
                     line_width_default='1',
                     opacity=0.8,
                     center=(-122.48, 37.83),
                     zoom=13,
                     below_layer='waterway-label',
                     height='300px')
viz5.show()